In [1]:
import os
import pandas as pd
import numpy as np
import scipy.stats
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.integrate import quad
from utils import generate_yyyymm, validate_uniqueness
import warnings
from scipy.optimize import fmin
from ipywidgets import *
import statsmodels.api as sm
from scipy.integrate import quad
from dateutil.relativedelta import relativedelta

In [2]:
# 사용자정의 함수
def summary(data, typ=0):
    if typ==0:
        count = len(data)
        mean = data.mean()
        std = data.std()
        min_ = data.min()
        max_ = data.max()
        q = list(np.quantile(data, q=[0.005, 0.025, 0.05, 0.25, 0.50, 0.75, 0.95, 0.975, 0.995]))
    elif typ==1:
        count = np.nan
        mean = data.mean()
        std = data.std()
        min_ = np.nan
        max_ = np.nan
        q = [data.ppf(0.005), data.ppf(0.025), data.ppf(0.05), data.ppf(0.25), data.ppf(0.50), data.ppf(0.75), data.ppf(0.95), data.ppf(0.975), data.ppf(0.995)]
    return pd.Series(data=[count, mean, std, min_] + q + [max_], index=['count', 'mean', 'std', 'min', '0.5%', '2.5%', '5%', '25%', '50%', '75%', '95%', '97.5%', '99.5%', 'max'])

def fit(data):
    dist_names = [
        "alpha","anglit","arcsine","beta","betaprime","bradford","burr","cauchy",
        "chi","chi2","cosine","dgamma","dweibull","erlang","expon","exponweib","exponpow",
        "f","fatiguelife","fisk","foldcauchy","foldnorm","frechet_r","frechet_l",
        "genlogistic","genpareto","genexpon","genextreme","gausshyper","gamma","gengamma",
        "genhalflogistic","gilbrat","gompertz","gumbel_r","gumbel_l","halfcauchy","halflogistic",
        "halfnorm","hypsecant","invgamma","invgauss","invweibull","johnsonsb","johnsonsu",
        "ksone","kstwobign","laplace","logistic","loggamma","loglaplace","lognorm","lomax",
        "maxwell","mielke","nakagami","ncx2","ncf","nct","norm","pareto","pearson3","powerlaw",
        "powerlognorm","powernorm","rdist","reciprocal","rayleigh","rice","recipinvgauss",
        "semicircular","t","triang","truncexpon","truncnorm","tukeylambda","uniform","vonmises",
        "wald","weibull_min","weibull_max","wrapcauchy",
    ]
    
    result = []
    for name in dist_names:
        dist = getattr(scipy.stats, name)
        params = dist.fit(data, floc=0)
        model = dist(*params[:-2], loc=params[-2], scale=params[-1])
        loglik = np.log(model.pdf(data)).sum()
        result.append([name, loglik])

    result = pd.DataFrame(result, columns=['분포명', '로그우도']) \
        .sort_values(by='로그우도', ascending=False) \
        .reset_index(drop=True)
    return result

In [3]:
# 환경설정
pd.set_option('mode.chained_assignment', None)
plt.style.use('default')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
if not any([s == 'result' for s in os.listdir('.')]): os.mkdir('result')
if not any([s == 'img' for s in os.listdir('.')]): os.mkdir('img')
warnings.filterwarnings('ignore')

In [ ]:
# 데이터 불러오기
파일명 = 'Package Insurance Policy'
작업일자 = '20201029'
실적 = pd.read_excel(f'data/{파일명}_실적_{작업일자}.xlsx', dtype={'증권번호': str, '계약FY년월': str})
사고 = pd.read_excel(f'data/{파일명}_사고_{작업일자}.xlsx', dtype={'사고접수번호': str, '증권번호': str, '계약FY년월': str, '사고발생일자': str, '사고접수일자': str})
증권정보 = pd.read_excel(f'data/{파일명}_증권정보_{작업일자}.xlsx', dtype={'증권번호': str, 'POOL구분코드': str, '계약UY년월': str, '계약자고객번호': str, '회계처리자사원번호': str, '목적물코드': str, '상품코드': str})
섹션코드 = pd.read_excel('data/섹션코드.xlsx', dtype={'섹션코드': str}) \
    .loc[lambda x: x.섹션코드.isin(증권정보.섹션코드)]
목적물코드 = pd.read_excel('data/목적물코드.xlsx', dtype={'목적물코드': str}) \
    .loc[lambda x: x.목적물코드.isin(증권정보.목적물코드)]
물건구분코드 = pd.read_excel('data/물건구분코드.xlsx', dtype={'물건구분코드': str}) \
    .loc[lambda x: x.물건구분코드.isin(증권정보.대표물건구분코드)]
보험사코드 = pd.read_excel('data/보험사코드.xlsx', dtype={'보험사코드': str}) \
    .loc[lambda x: x.보험사코드.isin(증권정보.주관보험사코드)]
화재업종코드 = pd.read_excel('data/화재업종코드.xlsx', dtype={'화재업종코드': str}) \
    .loc[lambda x: x.화재업종코드.isin(증권정보.대표화재업종코드)]
공동인수구분코드 = pd.read_excel('data/공동인수구분코드.xlsx', dtype={'공동인수구분코드': str}) \
    .loc[lambda x: x.공동인수구분코드.isin(증권정보.공동인수구분코드)]
갱신구분코드 = pd.read_excel('data/갱신구분코드.xlsx', dtype={'갱신구분코드': str}) \
    .loc[lambda x: x.갱신구분코드.isin(증권정보.갱신구분코드)]
사업단번호 = pd.read_excel('data/사업단번호.xlsx', dtype={'사업단번호': str})
# 판단요율_증권번호 = pd.read_excel('data/판단요율_증권번호.xlsx', dtype={'증권번호': str})
# 출재사유_재물 = pd.read_excel('data/UY1904_2003_출재사유_재물.xlsx', dtype={'증권번호': str})

In [ ]:
# UY 및 FY 범위
uy_min = 증권정보.query('계약UY년월 != " "')['계약UY년월'].min()
uy_max = 증권정보.query('계약UY년월 != " "')['계약UY년월'].max()
fy_min = 실적['계약FY년월'].min()
fy_max = 실적['계약FY년월'].max()
print(f'UY: {uy_min} ~ {uy_max}')
print(f'FY: {fy_min} ~ {fy_max}')

In [ ]:
# 변수 설정
FY시작년월, FY종료년월 = datetime(2015, 1, 1), datetime(2020, 9, 1)
UY시작년월, UY종료년월 = datetime(2015, 1, 1), datetime(2020, 9, 1)
UY년월 = pd.Series(generate_yyyymm((UY시작년월.year, UY시작년월.month), (UY종료년월.year, UY종료년월.month), 1))
FY년월 = pd.Series(generate_yyyymm((FY시작년월.year, FY시작년월.month), (FY종료년월.year, FY종료년월.month), 1))

In [ ]:
# 증권정보 가공
증권정보_가공 = 증권정보.query('POOL구분코드 == " "').query('계약UY년월 >= @UY시작년월.strftime("%Y%m")')
증권정보_가공['보험기간시작일자'] = 증권정보_가공.groupby('증권번호')['보험기간시작일자'].max()[증권정보_가공.증권번호].values
증권정보_가공['보험기간종료일자'] = 증권정보_가공.groupby('증권번호')['보험기간종료일자'].max()[증권정보_가공.증권번호].values
증권정보_가공['계약자고객번호'] = 증권정보_가공.groupby('증권번호').apply(lambda x: x.sort_values(by=['회계일자', '계약확정일자']).tail(1)['계약자고객번호'].values[0])[증권정보_가공.증권번호].values
증권정보_가공['회계처리사업단번호'] = 증권정보_가공.groupby('증권번호').apply(lambda x: x.sort_values(by=['회계일자', '계약확정일자']).tail(1)['회계처리사업단번호'].values[0])[증권정보_가공.증권번호].values
증권정보_가공['총보험가입금액'] = 증권정보_가공.groupby('증권번호')['총보험가입금액'].max()[증권정보_가공.증권번호].values
증권정보_가공.loc[lambda x: x.증권번호 == "120150624638", "대표화재업종코드"] = "0231103"
증권정보_가공.loc[lambda x: x.증권번호 == "120160472328", "대표화재업종코드"] = "1242A03"
증권정보_가공.loc[lambda x: x.증권번호 == "120160643927", "대표화재업종코드"] = "1304002"
증권정보_가공.loc[lambda x: x.증권번호 == "120170458412", "대표화재업종코드"] = "0901002"
증권정보_가공.loc[lambda x: x.증권번호 == "120170789844", "대표화재업종코드"] = "0479D03"
증권정보_가공.loc[lambda x: x.증권번호 == "120170860962", "대표화재업종코드"] = "0451103"
증권정보_가공.loc[lambda x: x.증권번호 == "120170940571", "대표화재업종코드"] = "1292003"
증권정보_가공.loc[lambda x: x.증권번호 == "120180628352", "대표화재업종코드"] = "0801002"
증권정보_가공.loc[lambda x: x.증권번호 == "120190646774", "대표화재업종코드"] = "1022003"
증권정보_가공.loc[lambda x: x.증권번호 == "120190712993", "대표화재업종코드"] = "1401102"
증권정보_가공.loc[lambda x: x.증권번호 == "120150624638", "대표물건구분코드"] = "03"
증권정보_가공.loc[lambda x: x.증권번호 == "120170789844", "대표물건구분코드"] = "03"
증권정보_가공.loc[lambda x: x.증권번호 == "120170860962", "대표물건구분코드"] = "03"
증권정보_가공.loc[lambda x: x.증권번호 == "120160554882", "주관보험사코드"] = "10"
증권정보_가공.loc[lambda x: x.증권번호 == "120160647858", "주관보험사코드"] = "10"
증권정보_가공.loc[lambda x: x.증권번호 == "120160648148", '계약UY년월'] = "201606" # 계약FY년월 < 계약UY년월 수정
증권정보_가공 = 증권정보_가공[['증권번호', '상품코드', '계약UY년월', '총보험가입금액', '보험기간시작일자', '보험기간종료일자', '대표화재업종코드', '대표물건구분코드', '주관보험사코드', '공동인수율', '공동인수구분코드', '갱신구분코드', '계약자고객번호', '회계처리사업단번호']].drop_duplicates()
if not 증권정보_가공['증권번호'].is_unique: raise Exception('증권번호 유일성 오류')
증권정보_가공 = 증권정보_가공.assign(
    보험기간시작일자 = lambda x: pd.to_datetime(x.보험기간시작일자, format='%Y%m%d'),
    보험기간종료일자 = lambda x: pd.to_datetime(x.보험기간종료일자, format='%Y%m%d'),
    보험기간 = lambda x: (x.보험기간종료일자 - x.보험기간시작일자).apply(lambda x: x.days)
)

# validate_uniqueness(증권정보_가공, '증권번호', '보험기간시작일자')
# validate_uniqueness(증권정보_가공, '증권번호', '보험기간종료일자')
# validate_uniqueness(증권정보_가공, '증권번호', '계약UY년월')
# validate_uniqueness(증권정보_가공, '증권번호', '대표화재업종코드')
# validate_uniqueness(증권정보_가공, '증권번호', '대표물건구분코드')
# validate_uniqueness(증권정보_가공, '증권번호', '주관보험사코드')
# validate_uniqueness(증권정보_가공, '증권번호', '공동인수율')
# validate_uniqueness(증권정보_가공, '증권번호', '공동인수구분코드')
# validate_uniqueness(증권정보_가공, '증권번호', '갱신구분코드')
# validate_uniqueness(증권정보_가공, '증권번호', '계약자고객번호')
# validate_uniqueness(증권정보_가공, '증권번호', '회계처리사업단번호')
# validate_uniqueness(증권정보_가공, '증권번호', '총보험가입금액')

In [ ]:
# 출재사유_패키지 = 출재사유_재물.query('상품코드 == "10503"') \
#     [['증권번호', '보유한도초과구분코드', '기준출재율(최종)', '보유한도초과구분코드', '의무출재 여부', '임의출재사유(택1)', '출재율 결정사유(택1)\n* 기준출재율 존재시 : 초과 출재사유']] \
#     .rename(columns={'출재율 결정사유(택1)\n* 기준출재율 존재시 : 초과 출재사유': '출재율결정사유', '의무출재 여부': '의무출재여부', '임의출재사유(택1)': '임의출재사유', '기준출재율(최종)': '기준출재율'})

# 실적 가공
실적_가공 = 증권정보_가공.merge(실적, on='증권번호', how='left')
유효증권번호 = 실적_가공.groupby(['증권번호', '총보험가입금액'])['원수보험료'].sum() \
    .reset_index().query('원수보험료 > 0 and 총보험가입금액 > 0').증권번호.unique()
증권정보_유효 = 증권정보_가공.query('증권번호 in @유효증권번호')
증권정보_무효 = 증권정보_가공.query('증권번호 not in @유효증권번호')
실적_가공 = 실적_가공.query('증권번호 in @유효증권번호') \
    .merge(화재업종코드, left_on='대표화재업종코드', right_on='화재업종코드', how='left') \
    .merge(물건구분코드, left_on='대표물건구분코드', right_on='물건구분코드', how='left') \
    .merge(보험사코드, left_on='주관보험사코드', right_on='보험사코드', how='left') \
    .merge(공동인수구분코드, on='공동인수구분코드', how='left') \
    .merge(갱신구분코드, on='갱신구분코드', how='left') \
    .merge(목적물코드, on='목적물코드', how='left') \
    .merge(섹션코드, on='섹션코드', how='left') \
    .merge(사업단번호, left_on='회계처리사업단번호', right_on='사업단번호', how='left') \
    .assign(
        UY = lambda x: x.계약UY년월.str.slice(0, 4),
        FY = lambda x: x.계약FY년월.str.slice(0, 4),
        UY분기 = lambda x: "'" + x.계약UY년월.str.slice(2, 4) + "." + ((x.계약UY년월.str.slice(4, 6).astype(int)-1)//3+1).astype(str) + "Q",
        FY분기 = lambda x: "'" + x.계약FY년월.str.slice(2, 4) + "." + ((x.계약FY년월.str.slice(4, 6).astype(int)-1)//3+1).astype(str) + "Q",
        경과분기 = lambda x: 4*(x.계약FY년월.str.slice(2, 4).astype(int) - x.계약UY년월.str.slice(2, 4).astype(int))+((x.계약FY년월.str.slice(4, 6).astype(int)-1)//3+1)-((x.계약UY년월.str.slice(4, 6).astype(int)-1)//3+1),
        경과년도 = lambda x: x.계약FY년월.str.slice(2, 4).astype(int) - x.계약UY년월.str.slice(2, 4).astype(int),
        화재업종대분류명 = lambda x: x.화재업종대분류명.fillna('구분없음'),
        물건구분명 = lambda x: x.물건구분명.fillna('구분없음'),
        보험사명 = lambda x: x.보험사명.fillna('구분없음'),
        공동인수구분명 = lambda x: x.공동인수구분명.fillna('구분없음'),
        갱신구분명 = lambda x: x.갱신구분명.fillna('구분없음'),
        목적물명 = lambda x: x.목적물명.fillna('구분없음'),
        섹션명 = lambda x: x.섹션명.fillna('구분없음'),
        회계처리사업단명 = lambda x: x.조직사업단명.fillna('구분없음'),
        회계처리본부명 = lambda x: x.조직본부명.fillna('구분없음'),
        회계처리부문명 = lambda x: x.조직부문명.fillna('구분없음'),
        당사보험가입금액 = lambda x: x.총보험가입금액*x.공동인수율/100,
        가입금액그룹 = lambda x: np.where(x.총보험가입금액 > 1.2e12, '대형물건', '소형물건'),
    ) \
    .drop(['대표화재업종코드', '대표물건구분코드', '주관보험사코드', '공동인수구분코드',
        '갱신구분코드', '회계처리사업단번호', '사업단번호', '보험사코드', '물건구분코드',
        '화재업종코드', '조직사업단명', '조직본부명', '조직부문명', '목적물코드', '섹션코드', '계약자고객번호'], axis=1)

In [ ]:
사고_가공 = 사고.loc[lambda x: x.증권번호.isin(유효증권번호)]
사고_가공['사고발생일자'] = 사고_가공.groupby('사고접수번호').apply(lambda x: x.sort_values(by='마감년월').tail(1)['사고발생일자'].values[0])[사고_가공.사고접수번호].values
사고_가공['사고접수일자'] = 사고_가공.groupby('사고접수번호').apply(lambda x: x.사고접수일자.min())[사고_가공.사고접수번호].values
사고_가공.loc[lambda x: x.사고접수번호 == "201804002796", "사고접수일자"] = "20180814"
사고_가공.loc[lambda x: x.사고접수번호 == "201604429941", "사고접수일자"] = "20161103"
사고_가공['증권번호'] = 사고_가공.groupby('사고접수번호').apply(lambda x: x.sort_values(by='계약보험가입금액').tail(1)['증권번호'].values[0])[사고_가공.사고접수번호].values
# 사고_가공['당사인수율'] = 사고_가공.groupby('사고접수번호').apply(lambda x: x.당사인수율.max())[사고_가공.사고접수번호].values
# 사고_가공['사고내용'] = 사고_가공.groupby('사고접수번호').apply(lambda x: x.sort_values(by='마감년월').tail(1)['사고내용'].values[0])[사고_가공.사고접수번호].values

# validate_uniqueness(사고_가공, '사고접수번호', '사고발생일자')
# validate_uniqueness(사고_가공, '사고접수번호', '사고접수일자')
# validate_uniqueness(사고_가공, '사고접수번호', '당사인수율')
# validate_uniqueness(사고_가공, '사고접수번호', '증권번호')

In [ ]:
보험금손조비 = 사고_가공.groupby('사고접수번호')[['당사지급보험금', '당사손해조사비']].sum().reset_index()
추산액 = 사고_가공.loc[lambda x: x.마감년월 == max(x.마감년월)].groupby('사고접수번호')['당사후기추산보험금'].sum().reset_index()
손해액 = 보험금손조비.merge(추산액, on='사고접수번호', how='outer') \
    .assign(당사후기추산보험금 = lambda x: x.당사후기추산보험금.fillna(0)) \
    .eval('원수손해액 = 당사지급보험금+당사손해조사비+당사후기추산보험금')
사고정보_가공 = 사고_가공[['사고접수번호', '사고발생일자', '사고접수일자', '증권번호', '당사인수율']].drop_duplicates()
사고정보_가공 = 사고정보_가공.merge(손해액, on='사고접수번호', how='left') \
    .query('원수손해액 > 0') \
    .merge(증권정보_가공, on='증권번호', how='left') \
    .assign(
        AY년월 = lambda x: "'" + x.사고발생일자.str.slice(2, 4) + "." + x.사고발생일자.str.slice(4, 6),
        AY = lambda x: x.사고발생일자.str.slice(0, 4),
        RY = lambda x: x.사고접수일자.str.slice(0, 4),
        UY = lambda x: x.계약UY년월.str.slice(0, 4),
        AY분기 = lambda x: "'" + x.사고발생일자.str.slice(2, 4) + "." + ((x.사고발생일자.str.slice(4, 6).astype(int)-1)//3+1).astype(str) + "Q",
        RY분기 = lambda x: "'" + x.사고접수일자.str.slice(2, 4) + "." + ((x.사고접수일자.str.slice(4, 6).astype(int)-1)//3+1).astype(str) + "Q",
        UY분기 = lambda x: "'" + x.계약UY년월.str.slice(2, 4) + "." + ((x.계약UY년월.str.slice(4, 6).astype(int)-1)//3+1).astype(str) + "Q",
        가입금액그룹 = lambda x: np.where(x.총보험가입금액 > 1.2e12, '대형물건', '소형물건'),
        원수손해액_log = lambda x: np.log(x.원수손해액),
        보정원수손해액 = lambda x: x.원수손해액/(x.공동인수율/100),
        보정원수손해액_log = lambda x: np.log(x.보정원수손해액),
        사고발생일자 = lambda x: pd.to_datetime(x.사고발생일자, format='%Y%m%d'),
        사고접수일자 = lambda x: pd.to_datetime(x.사고접수일자, format='%Y%m%d'),
        접수지연 = lambda x: (x.사고접수일자-x.사고발생일자).apply(lambda y: y.days),
    )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 4))

접수지연_당사_데이터 = 사고정보_가공.query('주관보험사코드 == "13"')['접수지연'] + 0.5
접수지연_타사_데이터 = 사고정보_가공.query('주관보험사코드 != "13"')['접수지연'] + 0.5

dist = getattr(scipy.stats, 'johnsonsu')
dist2 = getattr(scipy.stats, 'johnsonsu')
접수지연_당사_모수 = dist.fit(접수지연_당사_데이터, floc=0)
접수지연_타사_모수 = dist2.fit(접수지연_타사_데이터, floc=0)
model3 = dist(*접수지연_당사_모수[:-2], loc=접수지연_당사_모수[-2], scale=접수지연_당사_모수[-1])
model4 = dist2(*접수지연_타사_모수[:-2], loc=접수지연_타사_모수[-2], scale=접수지연_타사_모수[-1])

접수지연_당사_데이터.plot(kind='hist', bins=200, label='당사단독/당사간사', ax=ax, density=True, alpha=0.75)
접수지연_타사_데이터.plot(kind='hist', bins=200, label='타사간사', ax=ax, density=True, alpha=0.75)

x_min, x_max = 0, 접수지연_타사_데이터.max()
x_range = np.linspace(x_min, x_max, 1000)
y1_range = model3.pdf(x_range)
y2_range = model4.pdf(x_range)
ax.plot(x_range, y1_range, color='darkslategrey')
ax.plot(x_range, y2_range, color='darkslategrey')
ax.fill_between(x_range, 0, y1_range, color='darkslategrey', alpha=0.8, label='모델')
ax.fill_between(x_range, 0, y2_range, color='darkslategrey', alpha=0.8)

ax.set_title('패키지보험 공동인수구분별 접수지연 도수분포표', fontsize=15)
ax.set_xlabel('접수지연(일)', fontsize=15)
ax.set_ylabel('밀도', fontsize=15)
ax.tick_params(rotation=0, labelsize=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim([0, 500])
ax.legend()
plt.tight_layout()
plt.savefig('img/접수지연')
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 4), sharex=True)

사고건수 = 사고정보_가공.assign(당사구분 = lambda x: np.where(x.주관보험사코드 == "13", "당사", "타사")) \
    .pivot_table(index=['AY', 'AY분기'], columns=['당사구분', '가입금액그룹'], values='사고접수번호', aggfunc=len, fill_value=0) \
    .reset_index()
사고건수['경과분기'] = np.arange(len(사고건수['AY분기']))[::-1]
사고건수['사고보정계수_당사'] = 사고건수['경과분기'].apply(lambda i: 90/quad(model3.cdf, 90*i, 90*(i+1))[0])
사고건수['사고보정계수_타사'] = 사고건수['경과분기'].apply(lambda i: 90/quad(model4.cdf, 90*i, 90*(i+1))[0])
사고건수['사고건수_대형물건'] = 사고건수['당사']['대형물건']+사고건수['타사']['대형물건']
사고건수['사고건수_소형물건'] = 사고건수['당사']['소형물건']+사고건수['타사']['소형물건']
사고건수['보정사고건수_대형물건'] = 사고건수['당사']['대형물건']*사고건수['사고보정계수_당사']+사고건수['타사']['대형물건']*사고건수['사고보정계수_타사']
사고건수['보정사고건수_소형물건'] = 사고건수['당사']['소형물건']*사고건수['사고보정계수_당사']+사고건수['타사']['소형물건']*사고건수['사고보정계수_타사']
사고건수 = 사고건수.drop(['당사', '타사'], axis=1)
사고건수.columns = [x[0] for x in 사고건수.columns]
익스포져 = 실적_가공.pivot_table(index='FY분기', columns='가입금액그룹', values='원수경과보험료', aggfunc=lambda x: x.sum()/1e8, fill_value=0) \
    .rename(columns={'대형물건': '원수경과보험료_대형물건', '소형물건': '원수경과보험료_소형물건'})
사고건수 = 사고건수.merge(익스포져, left_on='AY분기', right_on='FY분기', how='outer') \
    .eval('보정사고건수_100억원당_대형물건 = 보정사고건수_대형물건/원수경과보험료_대형물건*100') \
    .eval('보정사고건수_100억원당_소형물건 = 보정사고건수_소형물건/원수경과보험료_소형물건*100')

df1 = 사고건수.groupby('AY분기')['보정사고건수_소형물건'].sum()
df2 = 사고건수.groupby('AY분기')['사고건수_소형물건'].sum()
df1.plot(kind='bar', ax=ax[0], label='소형물건(보정)', alpha=0.5)
df2.plot(kind='bar', ax=ax[0], label='소형물건')
df3 = 사고건수.groupby('AY분기')['보정사고건수_대형물건'].sum()
df4 = 사고건수.groupby('AY분기')['사고건수_대형물건'].sum()
df3.plot(kind='bar', ax=ax[1], label='대형물건(보정)', alpha=0.5)
df4.plot(kind='bar', ax=ax[1], label='대형물건')

ax[0].set_ylim([0, 900]); ax[1].set_ylim([0, 900])
ax[0].set_xlabel('AY', fontsize=12)
ax[0].set_ylabel('사고건수(건)', fontsize=12); ax[1].set_ylabel('사고건수(건)', fontsize=12)
ax[0].spines['top'].set_visible(False); ax[1].spines['top'].set_visible(False)
ax[0].spines['left'].set_visible(False); ax[1].spines['left'].set_visible(False)
ax[0].spines['right'].set_visible(False); ax[1].spines['right'].set_visible(False)
ax[0].legend(loc=2); ax[1].legend(loc=2)
for i in range(len(df1)):
    ax[0].text(i-0.48, df1[i]+20, f'{df1[i]:,.0f}'.center(3), weight='bold')
    ax[1].text(i-0.48, df3[i]+20, f'{df3[i]:,.0f}'.center(3), weight='bold')
plt.tight_layout()
plt.savefig('img/사고건수')
plt.show()

In [ ]:
print(사고건수['원수경과보험료_대형물건'].sum(), 사고건수['원수경과보험료_소형물건'].sum())

In [ ]:
print(사고건수['사고건수_대형물건'].sum(), 사고건수['사고건수_대형물건'].mean())
print(사고건수['사고건수_소형물건'].sum(), 사고건수['사고건수_소형물건'].mean())
print(사고건수['보정사고건수_대형물건'].sum(), 사고건수['보정사고건수_대형물건'].mean())
print(사고건수['보정사고건수_소형물건'].sum(), 사고건수['보정사고건수_소형물건'].mean())

In [ ]:
빈도_대형물건 = 사고건수['보정사고건수_대형물건'].sum()/사고건수['원수경과보험료_대형물건'].sum()
빈도_소형물건 = 사고건수['보정사고건수_소형물건'].sum()/사고건수['원수경과보험료_소형물건'].sum()

In [ ]:
dist = getattr(scipy.stats, 'poisson')
익스포져_대형물건 = 600
익스포져_소형물건 = 600
익스포져 = 익스포져_대형물건+익스포져_소형물건
freq1 = dist(익스포져_대형물건*빈도_대형물건)
freq2 = dist(익스포져_소형물건*빈도_소형물건)

In [ ]:
p*model1.mean()+(1-p)*model2.mean()

In [ ]:
num_sim = 1000
n1 = freq1.rvs(num_sim)
n2 = freq2.rvs(num_sim)
result = []
for k1, k2 in zip(n1, n2):
    총손해액_대형물건 = 0
    총손해액_소형물건 = 0
    for _ in range(k1):
        loss = np.exp(model1.rvs()) if np.random.random() < p else np.exp(model2.rvs())
        총손해액_대형물건 += loss/1e8
    for _ in range(k2):
        loss = np.exp(model1.rvs()) if np.random.random() < p else np.exp(model2.rvs())
        총손해액_소형물건 += loss/1e8
    result.append([k1, k2, 총손해액_소형물건, 총손해액_대형물건])

In [ ]:
np.quantile(np.array([x[2]/익스포져_대형물건 for x in result]), 0.995)
np.quantile(np.array([x[3]/익스포져_소형물건 for x in result]), 0.995)

In [ ]:
plt.hist([x[2]/익스포져_소형물건*100 for x in result], bins=500, density=True, alpha=0.5, label='소형물건')
plt.hist([x[3]/익스포져_대형물건*100 for x in result], bins=500, density=True, alpha=0.5, label='대형물건')
plt.hist([(x[2]+x[3])/익스포져*100 for x in result], bins=500, density=True, alpha=0.5, label='전체')
plt.xlim([0, 200])
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 4), sharex=True)

df1 = 사고건수['원수경과보험료_소형물건']
df1.plot(kind='bar', ax=ax[0], label='소형물건')
df2 = 사고건수['원수경과보험료_대형물건']
df2.plot(kind='bar', ax=ax[1], label='대형물건')

# ax[0].set_ylim([0, 250]); ax[1].set_ylim([0, 250])
ax[0].set_xlabel('FY', fontsize=12)
ax[0].set_ylabel('원수경과보험료(억원)', fontsize=12); ax[1].set_ylabel('원수경과보험료(억원)', fontsize=12)
ax[0].spines['top'].set_visible(False); ax[1].spines['top'].set_visible(False)
ax[0].spines['left'].set_visible(False); ax[1].spines['left'].set_visible(False)
ax[0].spines['right'].set_visible(False); ax[1].spines['right'].set_visible(False)
ax[0].legend(loc=2); ax[1].legend(loc=2)
for i in range(len(df1)):
    ax[0].text(i-0.48, df1[i]+10, f'{df1[i]:,.0f}'.center(3), weight='bold')
    ax[1].text(i-0.48, df2[i]+10, f'{df2[i]:,.0f}'.center(3), weight='bold')
plt.tight_layout()
plt.savefig('img/원수경과보험료')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

df = 실적_가공.pivot_table(index=['UY', 'UY분기'], columns='가입금액그룹', values='원수보험료', aggfunc=lambda x: x.sum()/1e8) \
    .eval('전체 = 대형물건 + 소형물건').reset_index()
UY19원수보험료_3분기 = df.loc[lambda x: x.UY분기.isin(['\'19.1Q', '\'19.2Q', '\'19.3Q']) ,'전체'].sum()
UY20원수보험료_3분기 = df.loc[lambda x: x.UY분기.isin(['\'20.1Q', '\'20.2Q', '\'20.3Q']) ,'전체'].sum()
성장률 = UY20원수보험료_3분기/UY19원수보험료_3분기-1
df.loc[df.index[-1]+1] = ['2020', '\'20.4Q', df.loc[df.index[-4], '대형물건']*(1+성장률), df.loc[df.index[-4], '소형물건']*(1+성장률), df.loc[df.index[-4], '전체']*(1+성장률)]
df2 = df.groupby('UY')['전체'].sum()
df2.index = [f'\'{x[2:]:}' if x[2:] != "20" else f'\'{x[2:]:}(e)' for x in df2.index]
df2.plot(kind='bar', ax=ax, color='red', alpha=0.5)
df3 = df.groupby('UY')['대형물건'].sum()
df3.index = [f'\'{x[2:]:}' if x[2:] != "20" else f'\'{x[2:]:}(e)' for x in df3.index]
df3.plot(kind='bar', ax=ax, color='blue', alpha=0.5)
ax.set_title('패키지보험 UY별 원수보험료', fontsize=15)
ax.set_xlabel('UY', fontsize=15)
ax.set_ylabel('손해율', fontsize=15)
ax.tick_params(rotation=0, labelsize=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for i in range(len(df2)):
    ax.text(i-0.22, df2[i]+10, f'{df2[i]:,.0f}'.center(5), size=12, weight='bold')
ax.set_ylim([0, 1300])

plt.tight_layout()
plt.savefig('img/UY별원수보험료')
plt.show()
df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

df = 실적_가공.groupby('UY')[['원수경과보험료', '원수손해액']].sum().eval('원수손해율 = 원수손해액/원수경과보험료*100')['원수손해율']
df.plot(kind='line', marker='o', ax=ax)
ax.set_title('패키지보험 UY별 원수손해율', fontsize=15)
ax.set_xlabel('UY', fontsize=15)
ax.set_ylabel('손해율', fontsize=15)
ax.tick_params(rotation=0, labelsize=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for i in range(len(df)):
    ax.text(i-0.15, df[i]+5, f'{df[i]:,.0f}'.center(5), size=12, weight='bold')

plt.tight_layout()
plt.savefig('img/UY별손해율')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

로그_손해액_데이터 = 사고정보_가공['원수손해액_log'].copy()

def obj_fun(args):
    p, mu1, sigma1, mu2, sigma2 = args
    model1 = getattr(scipy.stats, 'norm')(mu1, sigma1)
    model2 = getattr(scipy.stats, 'norm')(mu2, sigma2)
    loglik = np.log(p*model1.pdf(로그_손해액_데이터)+(1-p)*model2.pdf(로그_손해액_데이터)).sum()
    return -loglik

x0 = (0.5, 13, 1.5, 14, 2.5)
p, mu1, sigma1, mu2, sigma2 = scipy.optimize.fmin(obj_fun, x0)

로그_손해액_대형물건_데이터 = 사고정보_가공.query('가입금액그룹 == "대형물건"')['원수손해액_log'].copy()
로그_손해액_소형물건_데이터 = 사고정보_가공.query('가입금액그룹 == "소형물건"')['원수손해액_log'].copy()
로그_손해액_대형물건_데이터.plot(kind='hist', density=True, bins=200, alpha=0.55, label='대형물건', ax=ax)
로그_손해액_소형물건_데이터.plot(kind='hist', density=True, bins=200, alpha=0.55, label='소형물건', ax=ax)

dist = getattr(scipy.stats, 'norm')
model1 = dist(mu1, sigma1)
model2 = dist(mu2, sigma2)
x_min, x_max = 로그_손해액_데이터.min(), 로그_손해액_데이터.max()
x_range = np.linspace(x_min, x_max, 1000)
y_range = p*model1.pdf(x_range)+(1-p)*model2.pdf(x_range)
ax.plot(x_range, y_range, color='darkslategrey')
ax.fill_between(x_range, 0, y_range, color='darkslategrey', alpha=0.7, label='모델(물건전체)')
ax.set_title('패키지보험 사고별 원수손해액 도수분포표', fontsize=15)
ax.set_xlabel('log(원수손해액)', fontsize=15)
ax.set_ylabel('확률밀도', fontsize=15)
ax.tick_params(rotation=0, labelsize=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_yticklabels([])
ax.legend()
# ax.text(6., 0.25, f'$\mu={param[-2]:,.1f}, \sigma={param[-1]:,.2f}$', fontsize=12)
# ax.text(6., 0.22, f'$E[X]={np.exp(param[-2]+0.5*param[-1]**2):,.0f}$', fontsize=12)

plt.tight_layout()
plt.savefig('img/원수손해액분포')
plt.show()

In [ ]:
대형물건_요약 = summary(사고정보_가공.query('가입금액그룹 == "대형물건"')['원수손해액'])
소형물건_요약 = summary(사고정보_가공.query('가입금액그룹 == "소형물건"')['원수손해액'])
전체_요약 = summary(사고정보_가공['원수손해액'])
요약 = pd.concat([대형물건_요약, 소형물건_요약, 전체_요약], axis=1) \
    .rename(columns={0: '대형물건', 1: '소형물건', 2: '전체'})
요약

In [ ]:
x = 4
f'x={x}'

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

# df = 실적_가공.groupby(['증권번호', 'UY', '당사보험가입금액'])['원수보험료'].sum().reset_index() \
#     .groupby('UY')[['원수보험료', '당사보험가입금액']].sum() \
#     .eval('요율 = 원수보험료/당사보험가입금액*100')
# df['요율'].plot(kind='line', marker='o', ax=ax[0], color='orangered')
# ax[0].set_title('패키지보험 UY별 요율', fontsize=15)
# ax[0].set_xlabel('UY', fontsize=15)
# ax[0].set_ylabel('요율(%)', fontsize=15)
# ax[0].tick_params(rotation=0, labelsize=15)
# ax[0].spines['top'].set_visible(False)
# ax[0].spines['right'].set_visible(False)
# for x in range(len(df)):
#     ax[0].text(x-0.2, df['요율'][x], f'{df["요율"][x]:,.3f}', color='white', fontsize=12, fontweight='bold', bbox={'pad': 0.3, 'color': 'orangered', 'boxstyle': 'circle'})

df = 실적_가공.query('총보험가입금액 > 2e10').groupby(['증권번호', 'UY', '공동인수율', '당사보험가입금액', '총보험가입금액', '가입금액그룹']) \
    [['원수보험료', '원수경과보험료', '원수손해액', '임의출재보험료', '임의출재경과보험료', '임의출재손해액', '임의출재수수료']].sum().reset_index() \
        .eval('조정원수보험료 = 원수보험료/공동인수율') \
        .eval('요율 = 원수보험료/당사보험가입금액*100') \
        .assign(총보험가입금액_log = lambda x: np.log(x.총보험가입금액)) \
        .assign(조정원수보험료_log = lambda x: np.log(x.조정원수보험료))
sns.scatterplot(x='총보험가입금액_log', y='조정원수보험료_log', hue='가입금액그룹', data=df, s=20, ax=ax)
X_train = sm.add_constant(df['총보험가입금액_log'])
y_train = df['조정원수보험료_log']
model = sm.OLS(y_train, X_train)
res = model.fit()
const, beta = res.params
sigma = res.mse_resid
X_pred = np.linspace(23.7, 33, 100)
y_pred = res.predict(sm.add_constant(np.linspace(23.7, 33, 100)))
plt.plot(X_pred, y_pred, color='black', alpha=1, linestyle='dashed', lw=3)
plt.plot(X_pred, y_pred+1.96*sigma, color='crimson', alpha=1, linestyle='dashed', lw=2)
plt.plot(X_pred, y_pred-1.96*sigma, color='crimson', alpha=1, linestyle='dashed', lw=2)
ax.set_title('패키지보험 가입금액별 보험료 산포도(UY\'15 ~ UY\'20.3Q)', fontsize=15)
ax.set_xlabel('log(가입금액(S))', fontsize=15)
ax.set_ylabel('log(보험료(P))', fontsize=15)
ax.tick_params(rotation=0, labelsize=15)
ax.set_xlim([23.6, 33])
ax.set_ylim([7.5, 20])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.text(28, 9.5, f"""
    $ln(P) = {beta:.2f}·ln(S) - {-const:.2f} + ε$\n\
    $\epsilon$ ~ $N(0, {sigma:,.2f}^2)$
""", size=15, weight='bold')
    
plt.tight_layout()
plt.savefig('img/요율')
plt.show()

In [ ]:
df.groupby(['UY', '가입금액그룹'])[['총보험가입금액', '조정원수보험료']].sum().eval('요율 = 조정원수보험료/총보험가입금액*100')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(7)*4)
inner_colors = cmap(np.array([1, 2, 5]))

지표 = '원수보험료'

df = 실적_가공.groupby('물건구분명')[[지표]].sum().sort_values(by=지표, ascending=False).reset_index()
물건순서 = df.iloc[:2, 0].tolist() + ['기타']
물건기타 = df.iloc[2:].index
df.loc[lambda x: x.index.isin(물건기타), '물건구분명'] = '기타'
df = df.groupby('물건구분명')[지표].sum()[물건순서]
df.plot(kind='pie', y='원수보험료', ax=ax[0][0], legend=None, radius=1,
        wedgeprops={'width': 1, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.6, counterclock=True, labeldistance=1.1)
ax[0][0].set_ylabel(None)
ax[0][0].set_title(f'물건구분별 {지표} 비중', fontsize=15)

df = 실적_가공.groupby('화재업종대분류명')[[지표]].sum().sort_values(by=지표, ascending=False).reset_index()
화재업종순서 = df.iloc[:6, 0].tolist() + ['기타']
화재업종기타 = df.iloc[6:].index
df.loc[lambda x: x.index.isin(화재업종기타), '화재업종대분류명'] = '기타'
df = df.groupby('화재업종대분류명')[지표].sum()[화재업종순서]
df.plot(kind='pie', y=지표, ax=ax[0][1], legend=None, radius=1,
        wedgeprops={'width': 1, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.6, counterclock=True, labeldistance=1.1)
ax[0][1].set_ylabel(None)
ax[0][1].set_title(f'업종별 {지표} 비중', fontsize=15)


df = 실적_가공.groupby('보험사명')[[지표]].sum().sort_values(by=지표, ascending=False).reset_index()
보험사명순서 = df.iloc[:6, 0].tolist() + ['기타']
보험사명기타 = df.iloc[6:, 0].tolist()
df = 실적_가공.assign(보험사명 = lambda x: np.where(x.보험사명.isin(보험사명기타), "기타", x.보험사명)) \
    .groupby('보험사명')[지표].sum()[보험사명순서]
df.plot(kind='pie', y=지표, ax=ax[1][0], legend=None, radius=1, colors=outer_colors,
        wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.75, counterclock=True, labeldistance=1.1)
df2 = 실적_가공 \
    .groupby('공동인수구분명')[지표].sum()[['당사간사', '당사단독', '타사간사']]
df2.plot(kind='pie', y=지표, ax=ax[1][0], legend=None, radius=1.2-0.6, colors=inner_colors,
          wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold'}, startangle=0, pctdistance=0.5, counterclock=True, labeldistance=.7)
ax[1][0].set_ylabel(None)
ax[1][0].set_title(f'공동인수구분별 {지표} 비중', fontsize=15)

df = 실적_가공.groupby('갱신구분명')[지표].sum()
df.plot(kind='pie', y=지표, ax=ax[1][1], legend=None, radius=1,
        wedgeprops={'width': 1, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.6, counterclock=True, labeldistance=1.1)
ax[1][1].set_ylabel(None)
ax[1][1].set_title(f'갱신구분별 {지표} 비중', fontsize=15)

fig.text(0.43, 0.45, f'총 원수보험료\n  {df.sum()/1e8:,.0f}억원', size=26, weight='bold')

plt.tight_layout()
plt.savefig(f'img/구분별 {지표} 비중')
plt.show()

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# cmap = plt.get_cmap("tab20c")
# outer_colors = cmap(np.arange(7)*4)
# inner_colors = cmap(np.array([1, 2, 5]))

# 지표 = '원수손해액'
# 물건구분 = '소형물건'

# df = 실적_가공.query('가입금액그룹 == @물건구분').groupby('물건구분명')[[지표]].sum().sort_values(by=지표, ascending=False).reset_index()
# 물건순서 = df.iloc[:2, 0].tolist() + ['기타']
# 물건기타 = df.iloc[2:].index
# df.loc[lambda x: x.index.isin(물건기타), '물건구분명'] = '기타'
# df = df.groupby('물건구분명')[지표].sum()[물건순서]
# df.plot(kind='pie', y='원수보험료', ax=ax[0][0], legend=None, radius=1,
#         wedgeprops={'width': 1, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
#           textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.6, counterclock=True, labeldistance=1.1)
# ax[0][0].set_ylabel(None)
# ax[0][0].set_title(f'물건구분별 {지표} 비중', fontsize=15)

# df = 실적_가공.query('가입금액그룹 == @물건구분').groupby('화재업종대분류명')[[지표]].sum().sort_values(by=지표, ascending=False).reset_index()
# 화재업종순서 = df.iloc[:6, 0].tolist() + ['기타']
# 화재업종기타 = df.iloc[6:].index
# df.loc[lambda x: x.index.isin(화재업종기타), '화재업종대분류명'] = '기타'
# df = df.groupby('화재업종대분류명')[지표].sum()[화재업종순서]
# df.plot(kind='pie', y=지표, ax=ax[0][1], legend=None, radius=1,
#         wedgeprops={'width': 1, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
#           textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.6, counterclock=True, labeldistance=1.1)
# ax[0][1].set_ylabel(None)
# ax[0][1].set_title(f'업종별 {지표} 비중', fontsize=15)


# df = 실적_가공.query('가입금액그룹 == @물건구분').groupby('보험사명')[[지표]].sum().sort_values(by=지표, ascending=False).reset_index()
# 보험사명순서 = df.iloc[:6, 0].tolist() + ['기타']
# 보험사명기타 = df.iloc[6:, 0].tolist()
# df = 실적_가공.query('가입금액그룹 == @물건구분').assign(보험사명 = lambda x: np.where(x.보험사명.isin(보험사명기타), "기타", x.보험사명)) \
#     .groupby('보험사명')[지표].sum()[보험사명순서]
# df.plot(kind='pie', y=지표, ax=ax[1][0], legend=None, radius=1, colors=outer_colors,
#         wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
#           textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.75, counterclock=True, labeldistance=1.1)
# df2 = 실적_가공.query('가입금액그룹 == @물건구분') \
#     .groupby('공동인수구분명')[지표].sum()[['당사간사', '당사단독', '타사간사']]
# df2.plot(kind='pie', y=지표, ax=ax[1][0], legend=None, radius=1.2-0.6, colors=inner_colors,
#           wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
#           textprops={'size': 13, 'weight': 'bold'}, startangle=0, pctdistance=0.5, counterclock=True, labeldistance=.7)
# ax[1][0].set_ylabel(None)
# ax[1][0].set_title(f'공동인수구분별 {지표} 비중', fontsize=15)

# df = 실적_가공.query('가입금액그룹 == @물건구분').groupby('갱신구분명')[지표].sum()
# df.plot(kind='pie', y=지표, ax=ax[1][1], legend=None, radius=1,
#         wedgeprops={'width': 1, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
#           textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.6, counterclock=True, labeldistance=1.1)
# ax[1][1].set_ylabel(None)
# ax[1][1].set_title(f'갱신구분별 {지표} 비중', fontsize=15)

# fig.text(0.43, 0.45, f'총 원수보험료\n  {df.sum()/1e8:,.0f}억원', size=26, weight='bold')

# plt.tight_layout()
# plt.savefig(f'img/({물건구분})구분별 {지표} 비중')
# plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

지표전체 = [
    '원수보험료', '원수경과보험료','원수손해액',
    '출재보험료', '출재경과보험료', '출재손해액', '출재수수료',
    '특약출재보험료', '특약출재경과보험료', '특약출재손해액', '특약출재수수료',
    '임의출재보험료', '임의출재경과보험료', '임의출재손해액', '임의출재수수료'
]

df = 실적_가공 \
    .assign(가입금액그룹 = lambda x: np.where(x.총보험가입금액 < 1.2e12, '1.2조 미만', '1.2조 이상')) \
    .groupby(['증권번호', '가입금액그룹'])[지표전체].sum().reset_index()
건수 = df.groupby('가입금액그룹').size()
df = df.groupby('가입금액그룹')[지표전체].sum()
df['건수'] = 건수
df = df.eval('출재수지손 = 출재경과보험료 - 출재손해액 + 출재수수료') \
    .eval('특약출재수지손 = 특약출재경과보험료 - 특약출재손해액 + 특약출재수수료') \
    .eval('임의출재수지손 = 임의출재경과보험료 - 임의출재손해액 + 임의출재수수료') \
    .eval('출재율 = 출재보험료/원수보험료*100') \
    .eval('특약출재율 = 특약출재보험료/원수보험료*100') \
    .eval('임의출재율 = 임의출재보험료/원수보험료*100') \
    .eval('원수손해율 = 원수손해액/원수경과보험료*100') \
    .eval('출재손해율 = 출재손해액/출재경과보험료*100') \
    .eval('특약출재손해율 = 특약출재손해액/특약출재경과보험료*100') \
    .eval('임의출재손해율 = 임의출재손해액/임의출재경과보험료*100') \
    .eval('출재수수료율 = 출재수수료/출재보험료*100') \
    .eval('특약출재수수료율 = 특약출재수수료/특약출재보험료*100') \
    .eval('임의출재수수료율 = 임의출재수수료/임의출재보험료*100')
지표목록 = ['원수손해율', '출재율', '출재손해율', '출재수수료율']
df['건수'].plot(kind='pie', ax=ax[0][0], legend=None, radius=1.2, colors=outer_colors,
        wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.75, counterclock=True, labeldistance=1.05)
ax[0][0].set_ylabel('')
ax[0][0].text(-0.25, -0.05, f"{df['건수'].sum():,.0f}건", size=18, weight='bold')
df['원수보험료'].plot(kind='pie', ax=ax[0][1], legend=None, radius=1.2, colors=outer_colors,
        wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.75, counterclock=True, labeldistance=1.05)
ax[0][1].set_ylabel('')
ax[0][1].text(-0.35, -0.05, f"{df['원수보험료'].sum()/1e8:,.0f}억원", size=18, weight='bold')

df2 = df[지표목록].reset_index().melt(id_vars='가입금액그룹')
sns.barplot(x='variable', y='value', data=df2, hue='가입금액그룹', ax=ax[1][0])
ax[1][0].set_ylabel('')
ax[1][0].set_xlabel('')
ax[1][0].spines['top'].set_visible(False)
ax[1][0].spines['right'].set_visible(False)
ax[1][0].spines['left'].set_visible(False)
for i in range(4):
    ax[1][0].text(i-0.33, df[지표목록[i]]['1.2조 미만']+2, f'{df[지표목록[i]]["1.2조 미만"]:,.1f}', size=15)
    ax[1][0].text(i+0.06, df[지표목록[i]]['1.2조 이상']+2, f'{df[지표목록[i]]["1.2조 이상"]:,.1f}', size=15)
ax[1][0].set_title('가입금액그룹별 보유·수익성 지표')
ax[1][0].set_ylim([0, 75])
    
df.plot(kind='pie', ax=ax[1][1], y='출재수지손', legend=None, radius=1.2, colors=outer_colors,
        wedgeprops={'width': .6, 'edgecolor': 'w'}, autopct=lambda pct: f'{pct:,.1f}',
          textprops={'size': 13, 'weight': 'bold', 'color': 'black'}, startangle=0, pctdistance=0.75, counterclock=True, labeldistance=1.05)
ax[1][1].text(-0.35, -0.05, f"{df['출재수지손'].sum()/1e8:,.0f}억원", size=18, weight='bold')
ax[1][1].set_ylabel('')
ax[1][1].set_title('가입금액그룹별 출재수지손 비중')

# plt.subplots_adjust(wspace=2, hspace=1)
plt.tight_layout()
plt.savefig('img/가입금액별')
plt.show()

In [ ]:
실적_가공.query('화재업종대분류명 == "창고시설"').groupby('UY')[['원수경과보험료', '원수손해액']].sum().reset_index().eval('원수손해율=원수손해액/원수경과보험료*100')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 5))

df = 실적_가공 \
    .assign(가입금액그룹 = lambda x: np.where(x.총보험가입금액 < 1.2e12, '1.2조 미만', '1.2조 이상')) \
    .groupby(['가입금액그룹', 'UY'])[['원수보험료', '원수경과보험료', '원수손해액']].sum() \
    .eval('원수손해율 = 원수손해액/원수경과보험료*100')['원수손해율'].reset_index()
sns.barplot(y='원수손해율', x='UY', hue='가입금액그룹', data=df, ax=ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params('x', rotation=45, labelsize=13)
ax.set_xlabel('UY분기', size=15)

plt.tight_layout()
plt.savefig('img/원수손해율')
plt.show()

In [ ]:
지표전체 = [
    '원수보험료', '원수경과보험료','원수손해액',
    '출재보험료', '출재경과보험료', '출재손해액', '출재수수료',
    '특약출재보험료', '특약출재경과보험료', '특약출재손해액', '특약출재수수료',
    '임의출재보험료', '임의출재경과보험료', '임의출재손해액', '임의출재수수료'
]

df = 실적_가공 \
    .groupby(['증권번호'])[지표전체].sum().reset_index()
건수 = df.shape[0]
df = df[지표전체].sum()
df['건수'] = 건수

df = df.to_frame().T.eval('출재수지손 = 출재경과보험료 - 출재손해액 + 출재수수료') \
    .eval('특약출재수지손 = 특약출재경과보험료 - 특약출재손해액 + 특약출재수수료') \
    .eval('임의출재수지손 = 임의출재경과보험료 - 임의출재손해액 + 임의출재수수료') \
    .eval('출재율 = 출재보험료/원수보험료*100') \
    .eval('특약출재율 = 특약출재보험료/원수보험료*100') \
    .eval('임의출재율 = 임의출재보험료/원수보험료*100') \
    .eval('원수손해율 = 원수손해액/원수경과보험료*100') \
    .eval('출재손해율 = 출재손해액/출재경과보험료*100') \
    .eval('특약출재손해율 = 특약출재손해액/특약출재경과보험료*100') \
    .eval('임의출재손해율 = 임의출재손해액/임의출재경과보험료*100') \
    .eval('출재수수료율 = 출재수수료/출재보험료*100') \
    .eval('특약출재수수료율 = 특약출재수수료/특약출재보험료*100') \
    .eval('임의출재수수료율 = 임의출재수수료/임의출재보험료*100') \
    .T[0]

In [ ]:
# 출력
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/{파일명}_분석결과_{now}.xlsx', 'xlsxwriter') as writer:
    실적_가공.to_excel(writer, '실적_가공', index=False)
    사고정보_가공.to_excel(writer, '사고정보_가공', index=False)